eda:
    -importation et afichage des 3 df
    -tentative de concatenation des 3 df
    -reparation des featur de l'annees 2019 et re-concatenation
    -traitement des type de donnés
    -traitement des données manquantes
    -Créer un dictionnaire de mappage pour renommer les valeurs
    -changer les valeurs objet en format numerique
    -recuperation des 10 colone qui on le plus de corelation avec la target "Classification"
ia stuff


<h2>eda</h2>

In [ ]:
import pandas as pd


importation et afichage des 3 df

In [ ]:

df18 = pd.read_csv('./data/data-2018-64df4976a53ca451602301.csv')
df19 = pd.read_csv('./data/data-2019-64df49821d485822963415.csv')
df20 = pd.read_csv('./data/data-2020-64df49899653e310510744.csv')
print('df 2018')
df18.sample(5)


In [ ]:
print('df 2019')
df19.sample(5)

In [ ]:
print('df 2020')
df20.sample(5)

<h3>

tentative de concatenation des 3 df

In [ ]:
df_concat = pd.concat([df18, df19, df20], ignore_index=True)
df_concat.sample(5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
heatmap = sns.heatmap(df_concat.isna(), cmap='coolwarm')
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=45)
plt.show()

reparation des featur de l'annees 2019 et re-concatenation

In [ ]:
df20 = df20.drop('Unnamed: 8', axis=1)
print(df18.columns.to_list())
print(df19.columns.to_list())
print(df20.columns.to_list())


In [ ]:
df19 = df19.rename(columns={'EC': 'E.C','CO_-2 ': 'CO3','HCO_ - ': 'HCO3','Cl -': 'Cl','F -': 'F','NO3- ': 'NO3 ','SO4-2': 'SO4', 'Na+': 'Na', 'K+': 'K', 'Ca+2': 'Ca', 'Mg+2': 'Mg'})
df_concat = pd.concat([df18, df19, df20], ignore_index=True)

In [ ]:
print(df18.columns.to_list())
print(df19.columns.to_list())
print(df20.columns.to_list())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
heatmap = sns.heatmap(df_concat.isna(), cmap='coolwarm')
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=45)
plt.show()

traitement des type de donnés

In [ ]:
df_concat['pH'] = df_concat['pH'].replace('8..05', '8.05')
df_concat['pH'] = df_concat['pH'].astype(float)
df_concat['pH'].dtype

traitement des données manquantes

colonne CO3

In [ ]:
print('pourcentage de donné manquante sur les 3 ans',df_concat['CO3'].isnull().sum() / len(df_concat['CO3']))
print('pourcentage de donné manquante en 2019',df19['CO3'].isnull().sum() / len(df19['CO3']))

In [ ]:
df_concat = df_concat.drop('CO3', axis=1)

colonne gwl


In [ ]:
df_concat['gwl'].value_counts()
df_concat['gwl'].fillna(df_concat['gwl'].mean(), inplace=True)

afichage des données manquantes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
heatmap = sns.heatmap(df_concat.isna(), cmap='coolwarm')
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=45)
plt.show()

Créer un dictionnaire de mappage pour renommer les valeurs


In [ ]:
selection = {
    'C2S1': 'Moderate',
    'C3S1': 'Poor',
    'C4S2': 'Bad',
    'C4S1': 'Bad',
    'C3S2': 'Poor',
    'C4S4': 'Bad',
    'C4S3': 'Bad',
    'C1S1': 'Good',
    'C3S4': 'Bad',
    'C3S3': 'Poor',
    'C2S2': 'Moderate'
}

df_concat['Classification'] = df_concat['Classification'].replace(selection)
df_concat = df_concat[df_concat['Classification'].isin(selection.values())]


changer les valeurs objet en format numerique

In [ ]:
objet_columns = df_concat.select_dtypes(include='object').columns
print(objet_columns)

In [ ]:
for cols in objet_columns:
    unique_values = df_concat[cols].unique()
    print(cols," : ",unique_values)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()

for element in objet_columns:
    df_concat[element]=le.fit_transform(df_concat[element])

In [ ]:
df_concat.info()

recuperation des 10 colone qui on le plus de corelation avec la target "Classification"

In [ ]:
correlation_matrix = df_concat.corr()

correlations_with_target = correlation_matrix['Classification'].abs()

top_correlations = correlations_with_target.sort_values(ascending=False)

top_features = top_correlations.head(11).index

top_features_df = pd.DataFrame({'Top Features': top_features})

clean_df = df_concat.loc[:, top_features]

print(top_features_df)
column_names = top_features_df['Top Features'].tolist()

df_concat=df_concat.drop(columns=df_concat.columns.difference(column_names))
df_concat.info()




<h2>ia stuff</h2>


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

X = df_concat.drop('Classification', axis=1)
y=df_concat['Classification']


<h2> model LeaveOneOut: </h2> 


In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score  
import numpy as np
from sklearn.neighbors import KNeighborsClassifier


accuracy_scores = []
loo = LeaveOneOut()
for train_index, test_index in loo.split(X):
    X_train, X_test = np.array(X.iloc[train_index]), np.array(X.iloc[test_index])
    y_train, y_test = np.array(y.iloc[train_index]), np.array(y.iloc[test_index])

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(X_train, y_train)
    y_pred = knn_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print("Moyenne de précision avec LOO:", average_accuracy)

<h2> model kFold:</h2> 

In [ ]:

from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

accuracy_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = np.array(X.iloc[train_index]), np.array(X.iloc[test_index])
    y_train, y_test = np.array(y.iloc[train_index]), np.array(y.iloc[test_index])

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(X_train, y_train)
    y_pred = knn_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print("Moyenne de précision avec KFold:", average_accuracy)

<h2>model train_test_split:</h2> 


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
# X représente les caractéristiques et y les étiquettes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

neigh = KNeighborsClassifier(n_neighbors=5, weights="uniform")
X_test=np.array(X_test)
y_test=np.array(y_test)
neigh.fit(X_train, y_train)
# neigh.predict(X_train)
# neigh.predict_proba(X_train)
neigh.score(X_test, y_test, sample_weight=None)

<h2> model random forest:</h2> 

grie d'evaluation

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, make_scorer

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 3],
    'min_samples_split': [2, 5]
}
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted', zero_division=1),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='weighted')
}

RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier()

grid_search = GridSearchCV(clf, param_grid, scoring=scoring, cv=kf, refit='accuracy')
grid_search.fit(X, y)
best_estimator = grid_search.best_estimator_
best_estimator.fit(X, y)  
y_pred = best_estimator.predict(X)  
confusion = confusion_matrix(y, y_pred)

afichage

In [ ]:
print("="*40)
print(grid_search.best_estimator_.__class__.__name__)
print("="*40)
print("Best parameters:", grid_search.best_params_)
print("="*40)
for metric_name, metric_scorer in scoring.items():
    metric_value = grid_search.cv_results_[f"mean_test_{metric_name}"][grid_search.best_index_]
    print(f"mean {metric_name}: {metric_value:.4f}")
print("="*40)
print("Confusion Matrix:")
print(confusion)
print("="*40)

plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, cmap='cool', fmt='d')
plt.title("Matrice de confusion totale")
plt.xlabel("Classe prédite")
plt.ylabel("Classe réelle")
plt.show()